<a href="https://colab.research.google.com/github/groheri/DataScientistToolbox/blob/master/YT_Email_Reply_Llama3_CrewAI_%2B_Groq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install crewai langchain-groq duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 801.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 867.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
!pip show crewai

Name: crewai
Version: 0.28.8
Summary: Cutting-edge framework for orchestrating role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks.
Home-page: 
Author: Joao Moura
Author-email: joao@crewai.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: appdirs, click, embedchain, instructor, langchain, openai, opentelemetry-api, opentelemetry-exporter-otlp-proto-http, opentelemetry-sdk, pydantic, python-dotenv, regex
Required-by: 


# The goal

Reply to a customer email
1. get the email
2. categorize this as a "sales", "custom enquiry", "off topic", "customer complaint"
3. use the category to research info needed for the reply
4. take the info and check if there is enough to answer the email
5. write a reply
6. check the reply
7. save the email

### Agents
- categorizer
- researcher (web for now would be internal RAG)
- email writer


### Tasks
- categorize email
- research the answer
- write the email

In [ ]:
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

## Utilities

In [ ]:
import json  # Import the JSON module to parse JSON strings
from langchain_core.agents import AgentFinish

agent_finishes  = []

import json
from typing import Union, List, Tuple, Dict
from langchain.schema import AgentFinish

call_number = 0

def print_agent_output(agent_output: Union[str, List[Tuple[Dict, str]], AgentFinish], agent_name: str = 'Generic call'):
    global call_number  # Declare call_number as a global variable
    call_number += 1
    with open("crew_callback_logs.txt", "a") as log_file:
        # Try to parse the output if it is a JSON string
        if isinstance(agent_output, str):
            try:
                agent_output = json.loads(agent_output)  # Attempt to parse the JSON string
            except json.JSONDecodeError:
                pass  # If there's an error, leave agent_output as is

        # Check if the output is a list of tuples as in the first case
        if isinstance(agent_output, list) and all(isinstance(item, tuple) for item in agent_output):
            print(f"-{call_number}----Dict------------------------------------------", file=log_file)
            for action, description in agent_output:
                # Print attributes based on assumed structure
                print(f"Agent Name: {agent_name}", file=log_file)
                print(f"Tool used: {getattr(action, 'tool', 'Unknown')}", file=log_file)
                print(f"Tool input: {getattr(action, 'tool_input', 'Unknown')}", file=log_file)
                print(f"Action log: {getattr(action, 'log', 'Unknown')}", file=log_file)
                print(f"Description: {description}", file=log_file)
                print("--------------------------------------------------", file=log_file)

        # Check if the output is a dictionary as in the second case
        elif isinstance(agent_output, AgentFinish):
            print(f"-{call_number}----AgentFinish---------------------------------------", file=log_file)
            print(f"Agent Name: {agent_name}", file=log_file)
            agent_finishes.append(agent_output)
            # Extracting 'output' and 'log' from the nested 'return_values' if they exist
            output = agent_output.return_values
            # log = agent_output.get('log', 'No log available')
            print(f"AgentFinish Output: {output['output']}", file=log_file)
            # print(f"Log: {log}", file=log_file)
            # print(f"AgentFinish: {agent_output}", file=log_file)
            print("--------------------------------------------------", file=log_file)

        # Handle unexpected formats
        else:
            # If the format is unknown, print out the input directly
            print(f"-{call_number}-Unknown format of agent_output:", file=log_file)
            print(type(agent_output), file=log_file)
            print(agent_output, file=log_file)


In [ ]:
from langchain_groq import ChatGroq

GROQ_LLM = ChatGroq(
            # api_key=os.getenv("GROQ_API_KEY"),
            model="llama3-70b-8192"
        )


## Tools

In [ ]:
from crewai import Crew, Agent, Task, Process
from langchain_community.tools import DuckDuckGoSearchRun

In [ ]:
from datetime import datetime
from random import randint
from langchain.tools import tool

In [ ]:
import json
import os

import requests
from langchain.tools import tool

import os
from crewai import Agent, Task, Crew, Process
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import load_tools

search_tool = DuckDuckGoSearchRun()


In [ ]:
search_tool.run("usb key price")

'The Feitian ePass K9 USB Security Key offers all the features expected from a security key at an affordable price. ... The USB-C key is best suited for businesses that want to implement physical ... Best security. 7. Kingston IronKey Keypad 200. View at Amazon. View at Amazon. View at Newegg. The best USB flash drive for security This ultra secure flash drive features USB 3.2 Gen 1 speeds and ... It usually costs about $20-$30 less than the faster Kingston DataTraveler Max, while still delivering 616.9 MB/s sequential read speed and 543.9 MB/s sequential writes in our CrystalDiskMark tests ... YubiKey 5 NFC USB-A. Best Premium Security Key. The YubiKey 5 provides the most comprehensive protocols of any security key out there, as well as some excellent additional features for those who are security conscious. $50 at Amazon. No USB-C connection. -. Bigger than Nano version. Manufactured by Yubico, the codeveloper of the FIDO U2F open authentication standard used by many security keys, t

## Agents

In [ ]:
class EmailAgents():
    def make_categorizer_agent(self):
        return Agent(
            role='Email Categorizer Agent',
            goal="""take in a email from a human that has emailed out company email address and categorize it \
            into one of the following categories: \
            price_equiry - used when someone is asking for information about pricing \
            customer_complaint - used when someone is complaining about something \
            product_enquiry - used when someone is asking for information about a product feature, benefit or service but not about pricing \\
            customer_feedback - used when someone is giving feedback about a product \
            off_topic when it doesnt relate to any other category """,
            backstory="""You are a master at understanding what a customer wants when they write an email and are able to categorize it in a useful way""",
            llm=GROQ_LLM,
            verbose=True,
            allow_delegation=False,
            max_iter=5,
            memory=True,
            step_callback=lambda x: print_agent_output(x,"Email Categorizer Agent"),
        )

    def make_researcher_agent(self):
        return Agent(
            role='Info Researcher Agent',
            goal="""take in a email from a human that has emailed out company email address and the category \
            that the categorizer agent gave it and decide what information you need to search for for the email writer to reply to \
            the email in a thoughtful and helpful way.
            If you DONT think a search will help just reply 'NO SEARCH NEEDED'
            If you dont find any useful info just reply 'NO USEFUL RESESARCH FOUND'
            otherwise reply with the info you found that is useful for the email writer
            """,
            backstory="""You are a master at understanding what information our email writer needs  to write a reply that \
            will help the customer""",
            llm=GROQ_LLM,
            verbose=True,
            max_iter=5,
            allow_delegation=False,
            memory=True,
            tools=[search_tool],
            step_callback=lambda x: print_agent_output(x,"Info Researcher Agent"),
        )

    def make_email_writer_agent(self):
        return Agent(
            role='Email Writer Agent',
            goal="""take in a email from a human that has emailed out company email address, the category \
            that the categorizer agent gave it and the research from the research agent and \
            write a helpful email in a thoughtful and friendly way.

            If the customer email is 'off_topic' then ask them questions to get more information.
            If the customer email is 'customer_complaint' then try to assure we value them and that we are addressing their issues.
            If the customer email is 'customer_feedback' then try to assure we value them and that we are addressing their issues.
            If the customer email is 'product_enquiry' then try to give them the info the researcher provided in a succinct and friendly way.
            If the customer email is 'price_equiry' then try to give the pricing info they requested.

            You never make up information. that hasn't been provided by the researcher or in the email.
            Always sign off the emails in appropriate manner and from Sarah the Resident Manager.
            """,
            backstory="""You are a master at synthesizing a variety of information and writing a helpful email \
            that will address the customer's issues and provide them with helpful information""",
            llm=GROQ_LLM,
            verbose=True,
            allow_delegation=False,
            max_iter=5,
            memory=True,
            step_callback=lambda x: print_agent_output(x,"Email Writer Agent"),
        )



## Tasks

In [ ]:
class EmailTasks():
    # Define your tasks with descriptions and expected outputs
    def categorize_email(self, email_content):
        return Task(
            description=f"""Conduct a comprehensive analysis of the email provided and categorize into \
            one of the following categories:
            price_equiry - used when someone is asking for information about pricing \
            customer_complaint - used when someone is complaining about something \
            product_enquiry - used when someone is asking for information about a product feature, benefit or service but not about pricing \\
            customer_feedback - used when someone is giving feedback about a product \
            off_topic when it doesnt relate to any other category \

            EMAIL CONTENT:\n\n {email_content} \n\n
            Output a single cetgory only""",
            expected_output="""A single categtory for the type of email from the types ('price_equiry', 'customer_complaint', 'product_enquiry', 'customer_feedback', 'off_topic') \
            eg:
            'price_enquiry' \
            """,
            output_file=f"email_category.txt",
            agent=categorizer_agent
            )

    def research_info_for_email(self, email_content):
        return Task(
            description=f"""Conduct a comprehensive analysis of the email provided and the category \
            provided and search the web to find info needed to respond to the email

            EMAIL CONTENT:\n\n {email_content} \n\n
            Only provide the info needed DONT try to write the email""",
            expected_output="""A set of bullet points of useful info for the email writer \
            or clear instructions that no useful material was found.""",
            context = [categorize_email],
            output_file=f"research_info.txt",
            agent=researcher_agent
            )

    def draft_email(self, email_content):
        return Task(
            description=f"""Conduct a comprehensive analysis of the email provided, the category provided\
            and the info provided from the research specialist to write an email. \

            Write a simple, polite and too the point email which will respond to the customer's email. \
            If useful use the info provided from the research specialist in the email. \

            If no useful info was provided from the research specialist the answer politely but don't make up info. \

            EMAIL CONTENT:\n\n {email_content} \n\n
            Output a single cetgory only""",
            expected_output="""A well crafted email for the customer that addresses their issues and concerns""",
            context = [categorize_email, research_info_for_email],
            agent=email_writer_agent,
            output_file=f"draft_email.txt",
            )

## Instantiate Agents and Tasks

In [ ]:
# EMAIL = """HI there, \n
# I am emailing to find out the current price of Bitcoin. \n

# Can you please help me/

# Thanks,
# John
# """

EMAIL = """HI there, \n
I am emailing to say that I had a wonderful stay at your resort last week. \n

I really appreaciate what your staff did

Thanks,
Paul
"""

EMAIL = """HI there, \n
I am emailing to say that the resort weather was way to cloudy and overcast. \n
I wanted to write a song called 'Here comes the sun but it never came'

What should be the weather in Arizona in April?

I really hope you fix this next time.

Thanks,
George
"""

EMAIL = """HI there, \n
Why can't I get to sing?

Thanks,
Ringo
"""

In [ ]:
##
agents = EmailAgents()
tasks = EmailTasks()

## Agents
categorizer_agent = agents.make_categorizer_agent()
researcher_agent = agents.make_researcher_agent()
email_writer_agent = agents.make_email_writer_agent()

## Tasks
categorize_email = tasks.categorize_email(EMAIL)
research_info_for_email = tasks.research_info_for_email(EMAIL)
draft_email = tasks.draft_email(EMAIL)


## Crew

In [ ]:
from crewai import Crew, Process

# Instantiate your crew with a sequential process
crew = Crew(
    agents=[categorizer_agent, researcher_agent, email_writer_agent],
    tasks=[categorize_email, research_info_for_email, draft_email],
    verbose=2,
    process=Process.sequential,
    full_output=True,
    share_crew=False,
    step_callback=lambda x: print_agent_output(x,"MasterCrew Agent")
)

In [ ]:
# Kick off the crew's work
results = crew.kickoff()


# Print the results
print("Crew Work Results:")
print(results)

# print(f"Categorize Email: {categorize_email.output}")
print(crew.usage_metrics)

 [DEBUG]: == Working Agent: Email Categorizer Agent
 [INFO]: == Starting Task: Conduct a comprehensive analysis of the email provided and categorize into             one of the following categories:
            price_equiry - used when someone is asking for information about pricing             customer_complaint - used when someone is complaining about something             product_enquiry - used when someone is asking for information about a product feature, benefit or service but not about pricing \
            customer_feedback - used when someone is giving feedback about a product             off_topic when it doesnt relate to any other category 
            EMAIL CONTENT:

 HI there, 

Why can't I get to sing?

Thanks,
Ringo
 


            Output a single cetgory only


> Entering new CrewAgentExecutor chain...
Thought: After carefully analyzing the email content, I understand that the sender, Ringo, is expressing frustration about not being able to sing, but it's unclear what t

In [ ]:
!cat /content/draft_email.txt

Dear Ringo,

Thank you for reaching out to us. I'm happy to help you with your concern. However, I'm not entirely sure what you mean by "Why can't I get to sing?" Could you please provide more context or clarify what you're trying to accomplish or what issue you're facing?

Your help in providing more information will enable me to better understand your concern and provide a more accurate response.

Looking forward to hearing back from you.

Best regards,
Sarah the Resident Manager

In [ ]:
!cat /content/email_category.txt

off_topic

In [ ]:
!cat /content/research_info.txt

NO USEFUL RESEARCH FOUND

In [ ]:
#Task

from crewai import Task

class PersonalizeEmailTask():
    def personalize_email(self, agent, recipient, email_template):
        return Task(
            description=f"""
                Personalize the template email for recipient using their information.

                - Name: {recipient['first_name']} {recipient['last_name']}
                - Email: {recipient['email']}
                - Bio: {recipient['bio']}
                - Last conversation: {recipient['last_conversation']}

                Important Info to consider:
                - When personalizing the email, only use one sentence from the bio or last conversation.
                    And make sure to incorporate it naturally into the email.  without going too much in to detail.
                - Make sure to keep the updated email roughly the same same length as the template email.

                The template email is as follows:

                ```
                {email_template}
                ```
            """,
            agent=agent,
            expected_output=f"Personalized email draft.",
            async_execution=True,
        )

    def ghostwrite_email(self, agent, draft_email, recipient):
        return Task(
            description=f"""
                Revise the draft email to adopt the following writing style.

                Writing Style:
                - Use a more informal, engaging, and slightly sales-oriented tone, mirroring ghost writer's final email communication style.
                - This approach prioritizes clear, direct communication while maintaining a friendly and approachable tone.
                - Use straightforward language, including phrases like "Hey [Name]!" to start emails or messages.
                - The tone will be optimistic and encouraging, aiming to build rapport and motivate action, while staying grounded in practical advice.

                Important Notes:
                - Do not use emojis.
            """,
            agent=agent,
            context=[draft_email],
            expected_output=f"A revised email draft in ghost writer's specified tone and style.",
            output_file=f"output/{recipient['first_name']}_{recipient['last_name']}.txt",
        )

In [ ]:
#agents
import os
from crewai import Agent
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI


class EmailPersonalizationAgents():
    def __init__(self):
        self.llm = ChatGroq(
            api_key=os.getenv("GROQ_API_KEY"),
            model="llama3-70b-8192"
        )

        # self.llm = ChatOpenAI(
        #     model="gpt-4-turbo-preview",
        # )

    def personalize_email_agent(self):
        return Agent(
            role="Email Personalizer",
            goal=f"""
                Personalize template emails for recipients using their information.

                Given a template email and recipient information (name, email, bio, last conversation),
                personalize the email by incorporating the recipient's details
                into the email while maintaining the core message and structure of the original email.
                This involves updating the introduction, body, and closing of the email to make
                it more personal and engaging for each recipient.
                """,
            backstory="""
                As an Email Personalizer, you are responsible for customizing template emails for individual recipients based on their information and previous interactions.
                """,
            verbose=True,
            llm=self.llm,
            max_iter=2,
        )

    def ghostwriter_agent(self):
        return Agent(
            role="Ghostwriter",
            goal=f"""
                Revise draft emails to adopt the Ghostwriter's writing style.

                Use an informal, engaging, and slightly sales-oriented tone, mirroring the Ghostwriter's final email communication style.
                """,
            backstory="""
                As a Ghostwriter, you are responsible for revising draft emails to match the Ghostwriter's writing style, focusing on clear, direct communication with a friendly and approachable tone.
                """,
            verbose=True,
            llm=self.llm,
            max_iter=2,
        )

In [ ]:
!mkdir output

In [ ]:
#main

import os
import time
import csv
from crewai import Crew
from langchain_groq import ChatGroq
# from agents import EmailPersonalizationAgents
# from tasks import PersonalizeEmailTask

# 0. Setup environment
# from dotenv import load_dotenv
# load_dotenv()

email_template = """
Hey [Name]!

Just a quick reminder that we have a Skool community where you can
join us for weekly coaching calls every Tuesday at 6 PM Eastern time.
The community is completely free and we're about to hit the 500
user milestone. We'd love to have you join us!

If you have any questions or need help with your projects,
this is a great place to connect with others and get support.

If you're enjoying the AI-related content, make sure to check out
some of the other videos on my channel. Don't forget to hit that
like and subscribe button to stay updated with the latest content.
Looking forward to seeing you in the community!

Best regards,
Brandon Hancock
"""

# 1. Create agents
agents = EmailPersonalizationAgents()

email_personalizer = agents.personalize_email_agent()
ghostwriter = agents.ghostwriter_agent()

# 2. Create tasks
tasks = PersonalizeEmailTask()

personalize_email_tasks = []
ghostwrite_email_tasks = []

# Path to the CSV file containing client information
csv_file_path = 'clients_small.csv'

# Open the CSV file
with open(csv_file_path, mode='r', newline='') as file:
    # Create a CSV reader object
    csv_reader = csv.DictReader(file)

    # Iterate over each row in the CSV file
    for row in csv_reader:
        # Access each field in the row
        recipient = {
            'first_name': row['first_name'],
            'last_name': row['last_name'],
            'email': row['email'],
            'bio': row['bio'],
            'last_conversation': row['last_conversation']
        }

        # Create a personalize_email task for each recipient
        personalize_email_task = tasks.personalize_email(
            agent=email_personalizer,
            recipient=recipient,
            email_template=email_template
        )

        # Create a ghostwrite_email task for each recipient
        ghostwrite_email_task = tasks.ghostwrite_email(
            agent=ghostwriter,
            draft_email=personalize_email_task,
            recipient=recipient
        )

        # Add the task to the crew
        personalize_email_tasks.append(personalize_email_task)
        ghostwrite_email_tasks.append(ghostwrite_email_task)


# Setup Crew
crew = Crew(
    agents=[
        email_personalizer,
        ghostwriter
    ],
    tasks=[
        *personalize_email_tasks,
        *ghostwrite_email_tasks
    ],
    max_rpm=29
)

# Kick off the crew
start_time = time.time()

results = crew.kickoff()

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Crew kickoff took {elapsed_time} seconds.")
print("Crew usage", crew.usage_metrics)



> Entering new CrewAgentExecutor chain...


> Finished chain.


> Entering new CrewAgentExecutor chain...
Thought: I need to revise the draft email to adopt the Ghostwriter's writing style.

Final Answer:
Hey there,

I hope this email finds you well. I wanted to reach out to you regarding the recent issue with our agent stopping due to an iteration limit or time limit.

I completely understand how frustrating this can be, and I'm here to help you get back on track. Our team is committed to providing you with the best possible experience, and I'd be happy to assist you in resolving this issue.

Could you please provide me with more details about the error message you're seeing or any steps you've taken so far to troubleshoot the issue? This will help me better understand the situation and provide a more accurate solution.

I'm confident that we can work together to find a solution that suits your needs. If you have any questions or concerns, please don't hesitate to reach out. I'm her

In [ ]:
results

"Hey there,\n\nI hope this email finds you well. I wanted to reach out to you regarding the recent issue with our agent stopping due to an iteration limit or time limit.\n\nI completely understand how frustrating this can be, and I'm here to help you get back on track. Our team is committed to providing you with the best possible experience, and I'd be happy to assist you in resolving this issue.\n\nCould you please provide me with more details about the error message you're seeing or any steps you've taken so far to troubleshoot the issue? This will help me better understand the situation and provide a more accurate solution.\n\nI'm confident that we can work together to find a solution that suits your needs. If you have any questions or concerns, please don't hesitate to reach out. I'm here to help.\n\nLooking forward to hearing back from you and resolving this issue.\n\nBest regards,\n[Your Name]"